# Workshop on Domain-Specific Lanugages for Performance-Portable Weather and Climate Models

## Session 2B: Advanced Concepts II

### 1. Transient diffusion

In this example we will discretize and solve a time-dependent diffusion problem. If you're mathematically inclined, the PDE solved is

$$
\begin{align}
\mathrm{Domain}: \qquad&\Omega = [0, 1]^2&\\
\mathrm{PDE}: \qquad&\frac{\partial u}{\partial t} = \Delta u, \quad&&\mathbf{x} \in \Omega\\
\mathrm{IC}:\qquad &u(\mathbf{x}, 0) = 0, &&\mathbf{x} \in \Omega\\
\mathrm{BCs}:\qquad &u(\mathbf{x}, t) = 1 &&\mathbf{x} \in \Omega^{\mathrm{top}},\quad \forall t \in [0, \infty)\\
&\nabla u \cdot \mathbf{n} = 0, &&\mathbf{x} \in \partial \Omega \backslash \Omega^{\mathrm{top}},\quad \forall t \in [0, \infty)
\end{align}
$$

where $\Delta u$ is approximated by the 5-point simple second-order horizontal Laplacian (introduced in [Session1A.1.ipynb](Session-1A.1.ipynb#Horizontal-Laplacian)) and the approximate solution is evolved using a forward-Euler scheme in time.

We will use a uniform finite difference grid of $N$ points, where the outer points of the array are the bounds of $\Omega$. These are the points where the boundary condition is applied, and these points are consumed with the Laplace operator.

GT4Py (at the moment) is desiged to work only with 3D arrays, so we will allocate storages that are a single point in the $K$ direction.

First, we start with importing packages, setting up the discretization, and defining a few helper functions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import gt4py.storage as gt_storage
from gt4py.gtscript import stencil, function, PARALLEL, computation, interval, Field

# NOTE: These are the new language elements!
from gt4py.gtscript import parallel, region, I, J, K

from tools import print_generated_code

backend = "numpy"
dtype = np.float64

num_cells_1d = 16
nhalo = 1
shape = (num_cells_1d + 2*nhalo, num_cells_1d + 2*nhalo, 1)

dx = 1.0 / (num_cells_1d - 1)
dt = dx * dx / (2 * 1) * 0.9 # Valid timestep for stability

def make_zeros(default_origin=(nhalo, nhalo, 0)) -> Field:
    """Return a zero'd gt4py storage for the given backend and shape"""
    return gt_storage.zeros(backend=backend, shape=shape, dtype=dtype, default_origin=default_origin)

# Plotting
def plot_solution(u):
    """Plot the solution."""
    x = np.linspace(0, 1, num_cells_1d)
    axes = np.meshgrid(x, x)
    plt.contourf(*axes, u.data[nhalo:-nhalo, nhalo:-nhalo, 0], 50)
    plt.colorbar()
    plt.show()

# GT4Py Laplace operator
@function
def laplace(field):
    """Computes (u_xx + u_yy)."""
    xd2 = -2 * field[0, 0, 0] + field[-1, 0, 0] + field[1, 0, 0]
    yd2 = -2 * field[0, 0, 0] + field[0, -1, 0] + field[0, 1, 0]
    return (xd2 + yd2) / (dx * dx)

a. First attempt: use GT4Py without regions

In [ ]:
@stencil(backend)
def take_timestep(u: Field[dtype]):
    with computation(PARALLEL), interval(...):
        u += dt * laplace(u)

# Create storage (and set IC implicitly)
u = make_zeros()
for n in range(10):
    # Left - Neumann condition (du/dn = 0)
    u[0, :, :] = u[1, :, :]
    # Right - Neumann condition (du/dn = 0)
    u[-1, :, :] = u[-2, :, :]
    # Bottom - Neumann condition (du/dn = 0)
    u[:, 0, :] = u[:, 1, :]
    # Top - Dirichlet condition (u = 0)
    u[:, -1, :] = 1.0
    
    # Update approx. solution
    take_timestep(u, origin=(nhalo, nhalo, 0))

plot_solution(u)

**Tip**: The `domain` argument is actually not required, since the maximum domain size can be determined by the extents in the calculation. Try omitting it! This is more along the lines of what GT4Py has in mind - you actually have to tell it very little so there is minimal room for making mistakes and accessing out of bounds.

**What went well:** we are able to solve the PDE!

**What could be improved:** we have to escape the DSL in order to apply the boundary conditions, which inhibits the compiler to optimize larger sections of code.

On top of that, this will incur greater latency (and reduced performance) than if they were part of the stencil. Imagine these numpy operations were on a GPU: API calls for memory copies will have to re-issue reads and writes from global memory - this takes literally wastes hundreds (or more) of cycles.

<div class="alert alert-block alert-info">
    <b> Now it's your turn: </b><br>
    <ul>
        <li style="margin-bottom: 10px">Can you take the simple stencil above and add regions for the boundary conditions?</li>
        <li style="margin-bottom: 10px">Use <code>print_generated_code</code> to look at the code. Can you spot the regions? It may be easier if you change the backend to "debug".</li>
    </ul>
</div>

In [ ]:
@stencil(backend)
def take_timestep(u: Field[dtype]):
    with computation(PARALLEL), interval(...):
        # HINT: Put boundary conditions here!
        # with parallel(region[...]):
        #     u[0, 0, 0] = ...

        u += dt * laplace(u)

# Create storage (and set IC implicitly)
u = make_zeros()
for n in range(100):
    take_timestep(u, origin=(1, 1, 0))

plot_solution(u)

If you'd like, you can look at the generated code!

In [ ]:
print_generated_code(take_timestep)

### 2. Bi-Laplacian operator on a cubed sphere grid

Applying the Laplace operator once on a cubed sphere accesses one ring of halo points, but no special cases need to occur in the finite difference stencil to account for corner effects since it does not read corner values.

However, in the case of a bi-Lapacian operator (4th order operator) the fist application takes place on an extended domain indexing into the "ghost" points at the corners that do not exist. Therefore the Laplace function called needs to handle these points.

The code below sets up the operator with the corner conditions on the cubed sphere grid. The diagram below shows the operator on a corner of the grid avoiding the non-existent points. This requires a different stencil at these points, so regions must to be used to avoid escaping the DSL.

In [ ]:
from gt4py.gtscript import stencil, function, PARALLEL, computation, interval, Field, parallel, region, I, J, K

backend = "debug"
num_cells_1d = 128
dx = 1.0 / (num_cells_1d - 1)

@function
def d2x(u):
    """Centered u_xx finite difference."""
    return (-2 * u[0, 0, 0] + u[-1, 0, 0] + u[1, 0, 0]) / (dx * dx)


@function
def d2y(u):
    """Centered u_yy finite difference."""
    return (-2 * u[0, 0, 0] + u[0, -1, 0] + u[0, 1, 0]) / (dx * dx)

@function
def lap_cube_cells(u):
    """Compute an application of the Laplace operator on the cubed sphere.
    
    This version is capable of legally being called twice without indexing
    nonexistent values.
    """
    u_next = d2x(u) + d2y(u)

    with parallel(region[I[0], J[0] - 1]):
        u_next = d2y(u) + (-2 * u[0, 0, 0] + u[1, 0, 0] + u[-1, 1, 0]) / (dx * dx)
    with parallel(region[I[-1], J[0] - 1]):
        u_next = d2y(u) + (-2 * u[0, 0, 0] + u[1, 0, 0] + u[1, 1, 0]) / (dx * dx)
    with parallel(region[I[0], J[-1] + 1]):
        u_next = d2y(u) + (-2 * u[0, 0, 0] + u[1, 0, 0] + u[-1, -1, 0]) / (dx * dx)
    with parallel(region[I[-1], J[-1] + 1]):
        u_next = d2y(u) + (-2 * u[0, 0, 0] + u[1, 0, 0] + u[1, -1, 0]) / (dx * dx)

    with parallel(region[I[0] - 1, J[0]]):
        u_next = d2x(u) + (-2 * u[0, 0, 0] + u[0, 1, 0] + u[1, -1, 0]) / (dx * dx)
    with parallel(region[I[-1] + 1, J[0]]):
        u_next = d2x(u) + (-2 * u[0, 0, 0] + u[0, 1, 0] + u[-1, -1, 0]) / (dx * dx)
    with parallel(region[I[0] - 1, J[-1]]):
        u_next = d2x(u) + (-2 * u[0, 0, 0] + u[0, 1, 0] + u[1, 1, 0]) / (dx * dx)
    with parallel(region[I[-1] + 1, J[-1]]):
        u_next = d2x(u) + (-2 * u[0, 0, 0] + u[0, 1, 0] + u[-1, 1, 0]) / (dx * dx)

    return u_next

If this is called only once, the regions are not activated so the code for them is not even included in the generated output. We can illustrate that by generating and looking at the code for a single call to this function:

In [ ]:
@stencil(backend)
def lap_tmp(u: Field[dtype]):
    with computation(PARALLEL), interval(...):
        u = lap_cube_cells(u)

print_generated_code(lap_tmp)

On the other hand, if the function is called twice, the values of `u_next` resulting from the first call are used in the second call, so the regions are used in the generated code:

In [ ]:
@stencil(backend)
def biharmonic(u: Field[dtype]):
    with computation(PARALLEL), interval(...):
        u1 = lap_cube_cells(u)
        u = lap_cube_cells(u1)

print_generated_code(biharmonic)

#### Distributed computation

What we've seen so far is that regions can be used to express computation relative to the stencil compute domain boundaries, but it can also be used to restrict computation to certain MPI ranks. AxisOffsets like `I[0]` can be compile-time externals in a stencil. If these are set to `None` and used in a region, that region is automatically excluded.

<div class="alert alert-block alert-info">
    <b> Now it's your turn: </b><br>
    Imagine you are editing <code>lap_cube_cells</code> for a 3x3 processor layout on each tile. How would it look? Familiarize yourself with the syntax for now. Tomorrow you will continue this example using fv3gfs-util to execute it on the cubed sphere.
</div>

### 3. Python Decorators

In [ ]:
import time

def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        func(*args, **kwargs)
        print("Time: " + str(time.perf_counter() - start))
    return wrapper

@timeit
def say_whee():
    time.sleep(0.5)
    print("Whee!")

say_whee()

This is identical to the following

In [ ]:
def say_whee():
    time.sleep(0.5)
    print("Whee!")

say_whee = timeit(say_whee)

say_whee()

<div class="alert alert-block alert-info">
    <b> Now it's your turn: </b><br>
    <ul>
        <li style="margin-bottom: 10px">Extend the example above to aggregate call time to the functions decorated by <code>timeit</code>, then print a list at the end of the program.</li>
        <li style="margin-bottom: 10px">If you want a challenge, try applying it to a stencil. Note that when chaining decorators, the top decorator is the outer one.</li>
    </ul>
    Hints: You can still reference outer scope variables inside wrapper, and the following snippet may be useful to get you started
</div>

In [ ]:
import time
import types

import numpy as np

import gt4py
import gt4py.storage as gt_storage
from gt4py.gtscript import stencil, function, PARALLEL, computation, interval, Field, parallel, region

backend = "numpy"
dtype = np.float64

num_cells_1d = 10
shape = (num_cells_1d + 2*nhalo, num_cells_1d + 2*nhalo, 1)

def make_zeros(default_origin=(nhalo, nhalo, 0)) -> Field:
    """Return a zero'd gt4py storage for the given backend and shape"""
    return gt_storage.zeros(backend=backend, shape=shape, dtype=dtype, default_origin=default_origin)

def get_repr(func) -> str:
    """Return a string representation of the function.

    This is rather involved, so it is provided for your convenience.
    """
    if isinstance(func, types.FunctionType):
        return func.__name__
    elif isinstance(func, gt4py.StencilObject):
        return func.definition_func.__name__ + "_" + func._gt_id_
    else:
        raise TypeError("Unrecognized type")
        
def timeit(func):
    def wrapper(*args, **kwargs):
        call_name = get_repr(func)
        # Fill in code here
        # ...
    return wrapper

# Apply @timeit to a function, call it a few times,
# and print the aggregate results at the end of the snippet.

If you really have extra time and want to learn, there is an internal timer in the stencil already that can be used for each call to it. To use it, pass a variable that is a dict to the ``exec_info`` keyword argument to a stencil call, it returns you the time as a key in the dictionary.

Let's demonstrate that on the transient diffusion example:

In [ ]:
from pprint import pprint

u = make_zeros()
exec_info = {}
take_timestep(u, origin=(1, 1, 0), exec_info=exec_info)

pprint(exec_info)